In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [45]:
india_daily = pd.read_csv('./Wheat Futures India Daily (2015-22).csv',index_col=0)
india_weekly = pd.read_csv('./Wheat Futures India Weekly (2012-22).csv',index_col=0)
india_monthly = pd.read_csv('./Wheat Futures India Monthly (2007-22).csv',index_col=0)

In [46]:
india_daily = india_daily.replace({',': '', 'K':'', '%':''}, regex=True).astype(float)
india_weekly = india_weekly.replace({',': '', 'K':'', '%':''}, regex=True).astype(float)
india_monthly = india_monthly.replace({',': '', 'K':'', '%':''}, regex=True).astype(float)

In [47]:
india_daily = india_daily[::-1]
india_weekly = india_weekly[::-1]
india_monthly = india_monthly[::-1]

dly_chng = [0]*len(india_daily.Price.values)
weekly_chng = [0]*len(india_weekly.Price.values)
monthly_chng = [0]*len(india_monthly.Price.values)

for i in range(1, len(india_daily.Price.values)):
    dly_chng[i] = round((india_daily.Price.values[i] - india_daily.Price.values[i-1])/india_daily.Price.values[i-1] * 100, 2)

for i in range(1, len(india_weekly.Price.values)):
    weekly_chng[i] = round((india_weekly.Price.values[i] - india_weekly.Price.values[i-1])/india_weekly.Price.values[i-1] * 100, 2)

for i in range(1, len(india_monthly.Price.values)):
    monthly_chng[i] = round((india_monthly.Price.values[i] - india_monthly.Price.values[i-1])/india_monthly.Price.values[i-1] * 100, 2)

In [48]:
india_daily['Daily Change (%)'] = dly_chng
india_weekly['Weekly Change (%)'] = weekly_chng
india_monthly['Monthly Change (%)'] = monthly_chng

india_daily['Daily Change (%) Squared'] = np.square(india_daily['Daily Change (%)'].values)
india_weekly['Weekly Change (%) Squared'] = np.square(india_weekly['Weekly Change (%)'].values)
india_monthly['Monthly Change (%) Squared'] = np.square(india_monthly['Monthly Change (%)'].values)

In [49]:
india_daily.drop('20-05-2015', inplace=True)
india_weekly.drop('08-04-2012', inplace=True)
india_monthly.drop('01-06-2007', inplace=True)

india_daily.drop(columns=['Change %'], inplace=True)
india_weekly.drop(columns=['Change %'], inplace=True)
india_monthly.drop(columns=['Change %'], inplace=True)

In [50]:
india_weekly

,Price,Open,High,Low,Vol.,Weekly Change (%),Weekly Change (%) Squared
Date,,,,,,,
15-04-2012,1181.0,1232.0,1232.0,1155.0,9.76,-4.14,17.1396
22-04-2012,1185.0,1205.0,1211.0,1165.0,26.43,0.34,0.1156
29-04-2012,1182.0,1193.0,1219.0,1168.0,21.71,-0.25,0.0625
06-05-2012,1172.0,1184.0,1196.0,1170.0,19.23,-0.85,0.7225
13-05-2012,1132.0,1171.0,1190.0,1111.0,25.56,-3.41,11.6281
...,...,...,...,...,...,...,...
20-03-2022,2307.0,2258.0,2258.0,2258.0,NaN,2.12,4.4944
27-03-2022,2255.0,2331.0,2331.0,2256.0,NaN,-2.25,5.0625
03-04-2022,2216.0,2217.0,2217.0,2217.0,NaN,-1.73,2.9929
